## 1. Data Loading & Setup
Change runtime to GPU to work with model

In [ ]:
import ast
import csv
import json
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 

In [ ]:
!pip install -U pandasql
from pandasql import sqldf

In [ ]:
!wget https://github.com/sharvi24/Fine-grained-Entity-Typing/raw/main/training.zip
!unzip training.zip

In [ ]:
!wget https://github.com/sharvi24/Fine-grained-Entity-Typing/raw/main/test.zip
!unzip test.zip

In [ ]:
!wget https://raw.githubusercontent.com/sharvi24/Fine-grained-Entity-Typing/main/ltf2sent.py

In [ ]:
!wget https://raw.githubusercontent.com/sharvi24/Fine-grained-Entity-Typing/main/mentions.tab

In [ ]:
!git clone https://github.com/shahraj81/rufes.git

In [ ]:
!gdown 1jfVpUyoRheTlXj54zvrzmhBH66kTf_Uf

Downloading...
From: https://drive.google.com/uc?id=1jfVpUyoRheTlXj54zvrzmhBH66kTf_Uf
To: /content/lstm_fet.tar.gz
100% 5.66k/5.66k [00:00<00:00, 10.1MB/s]


In [ ]:
!tar -xvf /content/lstm_fet.tar.gz

data.py
model.py
train.py
util.py


In [ ]:
# train_file = '/content/en.train.ds.json'
# test_file = '/content/en.test.ds.json'
# dev_file = '/content/en.dev.ds.json'

# embed_file = '/content/enwiki.cbow.100d.case.txt'
# max_epoch = 1
# batch_size = 1
# buffer_size = 1
# in data.py as well

In [ ]:
!gdown 1EwJ5iUELkJxojmLpgXKy0M4LOxTGX3su

Downloading...
From: https://drive.google.com/uc?id=1EwJ5iUELkJxojmLpgXKy0M4LOxTGX3su
To: /content/enwiki.cbow.100d.case.txt
100% 2.07G/2.07G [00:22<00:00, 90.7MB/s]


In [ ]:
!gdown 1sx3d9J5hFha_o7xWxw08EixracAAoc5g

Downloading...
From: https://drive.google.com/uc?id=1sx3d9J5hFha_o7xWxw08EixracAAoc5g
To: /content/wikipedia_downsampling.tar.gz
100% 326M/326M [00:03<00:00, 94.5MB/s]


In [ ]:
!tar -xvf /content/wikipedia_downsampling.tar.gz

en.train.ds.json
en.test.ds.json
en.dev.ds.json


## 2. Data Preparation for Modeling

### A. Training Data

In [ ]:
filename = "/content/training/annotations/edl.tab"
df = pd.read_csv(filename, sep='\t',header=None)
df.columns = ['system_run_ID', 'mention_ID','mention_string','mention_justification',"entity_ID","entity_types","mention_type",
              "confidence_val"]
df.head()              

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,mention_type,confidence_val
0,NIST,annotation-0,Farmers,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,U.S. farmers,PER;PER.Farmer,NOM,1.0
1,NIST,annotation-1,U.S.,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,United States,GPE;GPE.Country,NAM,1.0
2,NIST,annotation-2,farmers,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,U.S. farmers,PER;PER.Farmer,NOM,1.0
3,NIST,annotation-3,They,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,U.S. farmers,PER;PER.Farmer,PRO,1.0
4,NIST,annotation-4,Bloomberg,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,Bloomberg,ORG;ORG.CommercialOrganization,NAM,1.0


In [ ]:
df[['File','Token_place']] = df.mention_justification.str.split(":",expand=True)
df.head()

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,mention_type,confidence_val,File,Token_place
0,NIST,annotation-0,Farmers,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,U.S. farmers,PER;PER.Farmer,NOM,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,0-6
1,NIST,annotation-1,U.S.,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,United States,GPE;GPE.Country,NAM,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,118-121
2,NIST,annotation-2,farmers,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,U.S. farmers,PER;PER.Farmer,NOM,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,123-129
3,NIST,annotation-3,They,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,U.S. farmers,PER;PER.Farmer,PRO,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,311-314
4,NIST,annotation-4,Bloomberg,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,Bloomberg,ORG;ORG.CommercialOrganization,NAM,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,397-405


In [ ]:
df[['Token_start','Token_end']] = df.Token_place.str.split("-",expand=True)
df.head()

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,mention_type,confidence_val,File,Token_place,Token_start,Token_end
0,NIST,annotation-0,Farmers,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,U.S. farmers,PER;PER.Farmer,NOM,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,0-6,0,6
1,NIST,annotation-1,U.S.,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,United States,GPE;GPE.Country,NAM,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,118-121,118,121
2,NIST,annotation-2,farmers,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,U.S. farmers,PER;PER.Farmer,NOM,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,123-129,123,129
3,NIST,annotation-3,They,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,U.S. farmers,PER;PER.Farmer,PRO,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,311-314,311,314
4,NIST,annotation-4,Bloomberg,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,Bloomberg,ORG;ORG.CommercialOrganization,NAM,1.0,20120211_WAPO_a6be1e92-50de-11e1-bd4f-8a7d53f6...,397-405,397,405


In [ ]:
# Number of unique entitiy types in training data
s2 = []
for i in range(len(df)):
  s2.append(df['entity_types'][i].split(";"))
flat_list2 = [item for sublist in s2 for item in sublist]
len(set(flat_list2))  

210

### B. Test Data

In [ ]:
file = '/content/test/annotations/gold.tab'
df_test = pd.read_csv(file, sep='\t',header=None)
df_test.columns = ['system_run_ID', 'mention_ID','mention_string','mention_justification',"entity_ID","entity_types","mention_type",
              "confidence_val"]
df_test.head()              

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,mention_type,confidence_val
0,NIST,annotation-0,cars,20131127_WAPO_0587868c1675836f50b1f036c4ad5e0d...,hydrogen cars,VEH;VEH.WheeledVehicle;VEH.WheeledVehicle.Moto...,NOM,1.0
1,NIST,annotation-1,Toyota,20131127_WAPO_0587868c1675836f50b1f036c4ad5e0d...,Toyota,ORG;ORG.CommercialOrganization;ORG.CommercialO...,NAM,1.0
2,NIST,annotation-2,vehicles,20131127_WAPO_0587868c1675836f50b1f036c4ad5e0d...,electric vehicles,VEH;VEH.WheeledVehicle;VEH.WheeledVehicle.Moto...,NOM,1.0
3,NIST,annotation-3,cars,20131127_WAPO_0587868c1675836f50b1f036c4ad5e0d...,electric vehicles,VEH;VEH.WheeledVehicle;VEH.WheeledVehicle.Moto...,NOM,1.0
4,NIST,annotation-4,Tesla,20131127_WAPO_0587868c1675836f50b1f036c4ad5e0d...,Tesla,ORG;ORG.CommercialOrganization;ORG.CommercialO...,NAM,1.0


In [ ]:
mentions_file = '/content/mentions.tab'
df3 = pd.read_csv(mentions_file, sep='\t',header=None, engine='python',quoting=csv.QUOTE_NONE)
df3.columns =['mention_string','mention_justification',"mention_type"]
df3.head()

,mention_string,mention_justification,mention_type
0,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NAM
1,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NAM
2,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NOM
3,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NOM
4,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NAM


In [ ]:
## left join: merge mentions & gold test df
df2_unclean = pd.merge(df3, df_test, on='mention_justification', how="left")
df2_unclean.head()

,mention_string_x,mention_justification,mention_type_x,system_run_ID,mention_ID,mention_string_y,entity_ID,entity_types,mention_type_y,confidence_val
0,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NAM,NIST,annotation-0,O.J. Simpson,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,1.0
2,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NOM,NIST,annotation-1,knife,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,1.0
3,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NOM,NIST,annotation-2,knife,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,1.0
4,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NAM,NIST,annotation-3,O.J. Simpson,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,1.0


In [ ]:
df2 = df2_unclean[['system_run_ID','mention_ID','mention_string_x','mention_justification','entity_ID','entity_types','mention_type_x','confidence_val']].copy()
df2.rename(columns = {'mention_string_x':'mention_string','mention_type_x':'mention_type'}, inplace = True)

# Assigning same values to system_run_ID & confidence_val columns
df2 = df2.assign(system_run_ID='NIST')
df2 = df2.assign(confidence_val=1.0)

# Replacing NaN in entity_types with empty string
df2.entity_types = df2.entity_types.fillna('')

# Extra
# 1. If entity_ID is NaN: put mention_string value
# 2. If mention_ID is NaN: ???

df2.head()

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,mention_type,confidence_val
0,NIST,NaN,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NaN,,NAM,1.0
1,NIST,annotation-0,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,1.0
2,NIST,annotation-1,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,1.0
3,NIST,annotation-2,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,1.0
4,NIST,annotation-3,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,1.0


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9996 entries, 0 to 9995
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   system_run_ID          9996 non-null   object 
 1   mention_ID             8123 non-null   object 
 2   mention_string         9996 non-null   object 
 3   mention_justification  9996 non-null   object 
 4   entity_ID              8123 non-null   object 
 5   entity_types           9996 non-null   object 
 6   mention_type           9996 non-null   object 
 7   confidence_val         9996 non-null   float64
dtypes: float64(1), object(7)
memory usage: 702.8+ KB


In [ ]:
s = []
for i in range(len(df2)):
  s.append(df2['entity_types'][i].split(";"))
flat_list = [item for sublist in s for item in sublist]
len(set(flat_list)) 

214

### C. Reading ltf files for sentence information

In [ ]:
import os
from os import listdir
from os.path import isfile, join
mypath = "/content/training/data/ltf/"
onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles[0]

'/content/training/data/ltf/20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17.ltf.xml'

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse(onlyfiles[0])
root = tree.getroot()

xmlstr = ET.tostring(root, encoding='UTF-8', method='xml')

In [ ]:
from ltf2sent import ltf2sent, load_ltf
#load_ltf(xmlstr)
sent = []
for f in onlyfiles:
  tree = ET.parse(f)
  root = tree.getroot()
  xmlstr = ET.tostring(root, encoding='UTF-8', method='xml')
  sent.append(ltf2sent(xmlstr))
sent[0]  

[['20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17',
  'Social media killing the secret ballot?',
  0,
  38],
 ['20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17',
  'Tuesday, November 6, 2012 13:33:02 GMT-05:00',
  40,
  83],
 ['20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17',
  'One of every five registered voters will share how they voted online, according to a new report by the Pew Internet & American Life Project. The report, "Social Media and Voting," also found that nearly a third of registered voters have been encouraged to vote one way or the other by contacts on Facebook or Twitter -- a number that rises among voters under 30.',
  85,
  446],
 ['20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17',
  'If that sounds like a lot of tweets and Facebook posts, you\'re right. The word "election" has appeared in more than 1.5 million tweets today alone, according to Topsy, and in the past 24 hours, both President Obama and Mitt Romney\'s names have been tweeted several million times. The T

In [ ]:
# Flattening the list
res = [list(ele) for s in sent for ele in s]
res[0]

['20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17',
 'Social media killing the secret ballot?',
 0,
 38]

In [ ]:
df_sent = pd.DataFrame(res, columns = ['File', 'Sentence','start_char','end_char'])
df_sent.head()

,File,Sentence,start_char,end_char
0,20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17,Social media killing the secret ballot?,0,38
1,20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17,"Tuesday, November 6, 2012 13:33:02 GMT-05:00",40,83
2,20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17,One of every five registered voters will share...,85,446
3,20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17,If that sounds like a lot of tweets and Facebo...,448,925
4,20121106_WAPO_5a9fa4f916f2d637826c27d836aa0a17,"On Facebook, 2.8 million people in the United ...",927,1090


### D. Combining lft data with train data

In [ ]:
df_new = pd.merge(df, df_sent, on='File', how="left")
df_new.head()

NameError: ignored

In [ ]:
df_new['Token_start'] = df_new['Token_start'].astype('int')
df_new['Token_end'] = df_new['Token_end'].astype('int')
df_new.info()

In [ ]:
q = "SELECT * FROM df_new WHERE Token_start >= start_char AND Token_end <= end_char"
df_total = sqldf(q, globals())
df_total.head()

In [ ]:
df_clean = df_total[['mention_string','entity_ID','entity_types','mention_type','Token_start','Token_end','Sentence']].copy()
df_clean.head()

In [ ]:
# break sentence into list of tokens 
check = df_clean.loc[:, "Sentence"].apply(lambda x: word_tokenize(x))
df_clean['tokens'] = check
df_clean.head()

### E. Converting data to a format suitable for LSTM

In [ ]:
# drop columns
df_clean = df_clean.drop(columns=['Sentence', 'mention_type'])
df_clean.head()

,mention_string,entity_ID,entity_types,Token_start,Token_end,tokens
0,Farmers,U.S. farmers,PER;PER.Farmer,0,6,"[Farmers, to, plant, most, acres, since, 1984,..."
1,U.S.,United States,GPE;GPE.Country,118,121,"[U.S., farmers, will, plant, the, most, acres,..."
2,farmers,U.S. farmers,PER;PER.Farmer,123,129,"[U.S., farmers, will, plant, the, most, acres,..."
3,They,U.S. farmers,PER;PER.Farmer,311,314,"[They, will, sow, corn, ,, soybeans, and, whea..."
4,Bloomberg,Bloomberg,ORG;ORG.CommercialOrganization,397,405,"[They, will, sow, corn, ,, soybeans, and, whea..."


In [ ]:
df_clean.entity_types = df_clean.entity_types.str.split(";")
df_clean.head()

,mention_string,entity_ID,entity_types,Token_start,Token_end,tokens
0,Farmers,U.S. farmers,"[PER, PER.Farmer]",0,6,"[Farmers, to, plant, most, acres, since, 1984,..."
1,U.S.,United States,"[GPE, GPE.Country]",118,121,"[U.S., farmers, will, plant, the, most, acres,..."
2,farmers,U.S. farmers,"[PER, PER.Farmer]",123,129,"[U.S., farmers, will, plant, the, most, acres,..."
3,They,U.S. farmers,"[PER, PER.Farmer]",311,314,"[They, will, sow, corn, ,, soybeans, and, whea..."
4,Bloomberg,Bloomberg,"[ORG, ORG.CommercialOrganization]",397,405,"[They, will, sow, corn, ,, soybeans, and, whea..."


In [ ]:
df_clean.rename(columns={'mention_string': 'mention', 'entity_ID': 'mention_id', 'entity_types':'labels', 
                         'Token_start':'start','Token_end':'end'}, inplace=True)
df_clean.head()

,mention,mention_id,labels,start,end,tokens
0,Farmers,U.S. farmers,"[PER, PER.Farmer]",0,6,"[Farmers, to, plant, most, acres, since, 1984,..."
1,U.S.,United States,"[GPE, GPE.Country]",118,121,"[U.S., farmers, will, plant, the, most, acres,..."
2,farmers,U.S. farmers,"[PER, PER.Farmer]",123,129,"[U.S., farmers, will, plant, the, most, acres,..."
3,They,U.S. farmers,"[PER, PER.Farmer]",311,314,"[They, will, sow, corn, ,, soybeans, and, whea..."
4,Bloomberg,Bloomberg,"[ORG, ORG.CommercialOrganization]",397,405,"[They, will, sow, corn, ,, soybeans, and, whea..."


In [ ]:
df_anno = df_clean[["mention", "mention_id", "labels", "start", "end"]]
df_anno.head()

,mention,mention_id,labels,start,end
0,Farmers,U.S. farmers,"[PER, PER.Farmer]",0,6
1,U.S.,United States,"[GPE, GPE.Country]",118,121
2,farmers,U.S. farmers,"[PER, PER.Farmer]",123,129
3,They,U.S. farmers,"[PER, PER.Farmer]",311,314
4,Bloomberg,Bloomberg,"[ORG, ORG.CommercialOrganization]",397,405


In [ ]:
df_anno_dict = df_anno.to_dict(orient='records')
df_anno_dict[0]

{'mention': 'Farmers',
 'mention_id': 'U.S. farmers',
 'labels': ['PER', 'PER.Farmer'],
 'start': 0,
 'end': 6}

In [ ]:
comb_list = []
for i in range(len(df_clean)):
  combined_dict = {}
  combined_dict["tokens"] = df_clean.iloc[0,5]
  combined_dict["annotations"] = [df_anno_dict[0]]
  comb_list.append(combined_dict)

In [ ]:
comb_list[:2]

[{'tokens': ['Farmers',
   'to',
   'plant',
   'most',
   'acres',
   'since',
   '1984',
   ',',
   'prompted',
   'by',
   'higher',
   'crop',
   'prices'],
  'annotations': [{'mention': 'Farmers',
    'mention_id': 'U.S. farmers',
    'labels': ['PER', 'PER.Farmer'],
    'start': 0,
    'end': 6}]},
 {'tokens': ['Farmers',
   'to',
   'plant',
   'most',
   'acres',
   'since',
   '1984',
   ',',
   'prompted',
   'by',
   'higher',
   'crop',
   'prices'],
  'annotations': [{'mention': 'Farmers',
    'mention_id': 'U.S. farmers',
    'labels': ['PER', 'PER.Farmer'],
    'start': 0,
    'end': 6}]}]

In [ ]:
with open("rufes_train.json", "w") as outfile:
    json.dump(comb_list, outfile)

In [ ]:
import json
    
with open("sample.json", "w") as outfile:
    json.dump(combined_dict, outfile)

### F. Doing the same for Test data

In [ ]:
df2_expand = df2.copy()
df2_expand[['File','Token_place']] = df2_expand.mention_justification.str.split(":",expand=True)
df2_expand[['Token_start','Token_end']] = df2_expand.Token_place.str.split("-",expand=True)
df2_expand.head()

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,mention_type,confidence_val,File,Token_place,Token_start,Token_end
0,NIST,NaN,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NaN,,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,17-19,17,19
1,NIST,annotation-0,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,44-55,44,55
2,NIST,annotation-1,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,57-61,57,61
3,NIST,annotation-2,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,139-143,139,143
4,NIST,annotation-3,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,164-175,164,175


In [ ]:
mypath2 = "/content/test/data/ltf/"
onlyfiles2 = [join(mypath2, f) for f in listdir(mypath2) if isfile(join(mypath2, f))]
onlyfiles2[0]

'/content/test/data/ltf/20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8.ltf.xml'

In [ ]:
sent2 = []
for f in onlyfiles2:
  tree = ET.parse(f)
  root = tree.getroot()
  xmlstr = ET.tostring(root, encoding='UTF-8', method='xml')
  sent2.append(ltf2sent(xmlstr))
sent2[0]  

[['20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8',
  'Diplomatic fashion crisis averted: Michelle Obama wears Vera Wang to China state dinner',
  0,
  86],
 ['20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8',
  'Sunday, September 27, 2015 17:52:05 GMT-05:00',
  88,
  132],
 ['20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8',
  'It wasn’t exactly an apology or a mea culpa. Call it a diplomatic clarification. Or a gesture of goodwill to Seventh Avenue.',
  134,
  257],
 ['20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8',
  'For Friday’s state dinner in honor of Chinese president Xi Jinping and his wife Peng Liyuan, first lady Michelle Obama chose a custom mermaid gown from Vera Wang Collection. It was an inky black floor-length dress in silk crepe with a hand-draped skirt and organza sleeves. It exuded Hollywood glamour of the sort that has long defined the red carpet. And Obama’s hair, styled in gentle waves and cascading down her right cheek, underscored the dress’s mood of old-fashioned

In [ ]:
# Flattening the list
res2 = [list(ele) for s in sent2 for ele in s]
res2[0]

['20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8',
 'Diplomatic fashion crisis averted: Michelle Obama wears Vera Wang to China state dinner',
 0,
 86]

In [ ]:
df_sent2 = pd.DataFrame(res2, columns = ['File', 'Sentence','start_char','end_char'])
df_sent2.head()

,File,Sentence,start_char,end_char
0,20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8,Diplomatic fashion crisis averted: Michelle Ob...,0,86
1,20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8,"Sunday, September 27, 2015 17:52:05 GMT-05:00",88,132
2,20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8,It wasn’t exactly an apology or a mea culpa. C...,134,257
3,20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8,For Friday’s state dinner in honor of Chinese ...,259,742
4,20150927_WAPO_8c2039d3ef50ec7564c287952a6e0eb8,[The White House state dinner for China: Lots ...,744,830


In [ ]:
df_new2 = pd.merge(df2_expand, df_sent2, on='File', how="left")
df_new2.head()

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,mention_type,confidence_val,File,Token_place,Token_start,Token_end,Sentence,start_char,end_char
0,NIST,NaN,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NaN,,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,17-19,17,19,Reported lack of DNA is another dead end in O....,0,75
1,NIST,NaN,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NaN,,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,17-19,17,19,"Tuesday, March 15, 2016 09:28:15 GMT-05:00",77,118
2,NIST,NaN,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NaN,,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,17-19,17,19,The discovery of a knife found years ago at O....,120,315
3,NIST,NaN,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NaN,,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,17-19,17,19,It probably won’t.,317,334
4,NIST,NaN,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NaN,,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,17-19,17,19,TMZ reported Tuesday that the Los Angeles Poli...,336,685


In [ ]:
df_new2['Token_start'] = df_new2['Token_start'].astype('int')
df_new2['Token_end'] = df_new2['Token_end'].astype('int')
df_new2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142907 entries, 0 to 142906
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   system_run_ID          142907 non-null  object 
 1   mention_ID             116327 non-null  object 
 2   mention_string         142907 non-null  object 
 3   mention_justification  142907 non-null  object 
 4   entity_ID              116327 non-null  object 
 5   entity_types           142907 non-null  object 
 6   mention_type           142907 non-null  object 
 7   confidence_val         142907 non-null  float64
 8   File                   142907 non-null  object 
 9   Token_place            142907 non-null  object 
 10  Token_start            142907 non-null  int64  
 11  Token_end              142907 non-null  int64  
 12  Sentence               142907 non-null  object 
 13  start_char             142907 non-null  int64  
 14  end_char               142907 non-nu

In [ ]:
from pandasql import sqldf
q2 = "SELECT * FROM df_new2 WHERE Token_start >= start_char AND Token_end <= end_char"
df_total2 = sqldf(q2, globals())
df_total2.head()

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,mention_type,confidence_val,File,Token_place,Token_start,Token_end,Sentence,start_char,end_char
0,NIST,None,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,None,,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,17-19,17,19,Reported lack of DNA is another dead end in O....,0,75
1,NIST,annotation-0,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,44-55,44,55,Reported lack of DNA is another dead end in O....,0,75
2,NIST,annotation-1,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,57-61,57,61,Reported lack of DNA is another dead end in O....,0,75
3,NIST,annotation-2,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,139-143,139,143,The discovery of a knife found years ago at O....,120,315
4,NIST,annotation-3,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,1.0,20160315_WAPO_55c03f01631ae453d58744906df9ac5b,164-175,164,175,The discovery of a knife found years ago at O....,120,315


In [ ]:
df_clean2 = df_total2[['mention_string','entity_ID','entity_types','mention_type','Token_start','Token_end','Sentence']].copy()
df_clean2.head()

,mention_string,entity_ID,entity_types,mention_type,Token_start,Token_end,Sentence
0,DNA,None,,NAM,17,19,Reported lack of DNA is another dead end in O....
1,O.J. Simpson,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,44,55,Reported lack of DNA is another dead end in O....
2,knife,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,57,61,Reported lack of DNA is another dead end in O....
3,knife,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,139,143,The discovery of a knife found years ago at O....
4,O.J. Simpson,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,164,175,The discovery of a knife found years ago at O....


In [ ]:
# break sentence into list of tokens 
check2 = df_clean2.loc[:, "Sentence"].apply(lambda x: word_tokenize(x))
df_clean2['tokens'] = check2
df_clean2.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,mention_string,entity_ID,entity_types,mention_type,Token_start,Token_end,Sentence,tokens
0,DNA,None,,NAM,17,19,Reported lack of DNA is another dead end in O....,"[Reported, lack, of, DNA, is, another, dead, e..."
1,O.J. Simpson,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,44,55,Reported lack of DNA is another dead end in O....,"[Reported, lack, of, DNA, is, another, dead, e..."
2,knife,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,57,61,Reported lack of DNA is another dead end in O....,"[Reported, lack, of, DNA, is, another, dead, e..."
3,knife,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,NOM,139,143,The discovery of a knife found years ago at O....,"[The, discovery, of, a, knife, found, years, a..."
4,O.J. Simpson,O.J. Simpson,PER;PER.SituationalRole.Suspect,NAM,164,175,The discovery of a knife found years ago at O....,"[The, discovery, of, a, knife, found, years, a..."


In [ ]:
# drop columns
df_clean2 = df_clean2.drop(columns=['Sentence', 'mention_type'])
df_clean2.head()

,mention_string,entity_ID,entity_types,Token_start,Token_end,tokens
0,DNA,None,,17,19,"[Reported, lack, of, DNA, is, another, dead, e..."
1,O.J. Simpson,O.J. Simpson,PER;PER.SituationalRole.Suspect,44,55,"[Reported, lack, of, DNA, is, another, dead, e..."
2,knife,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,57,61,"[Reported, lack, of, DNA, is, another, dead, e..."
3,knife,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,139,143,"[The, discovery, of, a, knife, found, years, a..."
4,O.J. Simpson,O.J. Simpson,PER;PER.SituationalRole.Suspect,164,175,"[The, discovery, of, a, knife, found, years, a..."


In [ ]:
df_clean2.entity_types = df_clean2.entity_types.str.split(";")
df_clean2.head()

,mention_string,entity_ID,entity_types,Token_start,Token_end,tokens
0,DNA,None,[],17,19,"[Reported, lack, of, DNA, is, another, dead, e..."
1,O.J. Simpson,O.J. Simpson,"[PER, PER.SituationalRole.Suspect]",44,55,"[Reported, lack, of, DNA, is, another, dead, e..."
2,knife,knife,"[WEA, WEA.CloseCombatWeapon, WEA.CloseCombatWe...",57,61,"[Reported, lack, of, DNA, is, another, dead, e..."
3,knife,knife,"[WEA, WEA.CloseCombatWeapon, WEA.CloseCombatWe...",139,143,"[The, discovery, of, a, knife, found, years, a..."
4,O.J. Simpson,O.J. Simpson,"[PER, PER.SituationalRole.Suspect]",164,175,"[The, discovery, of, a, knife, found, years, a..."


In [ ]:
df_clean2.rename(columns={'mention_string': 'mention', 'entity_ID': 'mention_id', 'entity_types':'labels', 
                         'Token_start':'start','Token_end':'end'}, inplace=True)
df_clean2.head()

,mention,mention_id,labels,start,end,tokens
0,DNA,None,[],17,19,"[Reported, lack, of, DNA, is, another, dead, e..."
1,O.J. Simpson,O.J. Simpson,"[PER, PER.SituationalRole.Suspect]",44,55,"[Reported, lack, of, DNA, is, another, dead, e..."
2,knife,knife,"[WEA, WEA.CloseCombatWeapon, WEA.CloseCombatWe...",57,61,"[Reported, lack, of, DNA, is, another, dead, e..."
3,knife,knife,"[WEA, WEA.CloseCombatWeapon, WEA.CloseCombatWe...",139,143,"[The, discovery, of, a, knife, found, years, a..."
4,O.J. Simpson,O.J. Simpson,"[PER, PER.SituationalRole.Suspect]",164,175,"[The, discovery, of, a, knife, found, years, a..."


In [ ]:
df_anno2 = df_clean2[["mention", "mention_id", "labels", "start", "end"]]
df_anno2.head()

,mention,mention_id,labels,start,end
0,DNA,None,[],17,19
1,O.J. Simpson,O.J. Simpson,"[PER, PER.SituationalRole.Suspect]",44,55
2,knife,knife,"[WEA, WEA.CloseCombatWeapon, WEA.CloseCombatWe...",57,61
3,knife,knife,"[WEA, WEA.CloseCombatWeapon, WEA.CloseCombatWe...",139,143
4,O.J. Simpson,O.J. Simpson,"[PER, PER.SituationalRole.Suspect]",164,175


In [ ]:
df_anno_dict2 = df_anno2.to_dict(orient='records')
df_anno_dict2[0]

{'mention': 'DNA', 'mention_id': None, 'labels': [''], 'start': 17, 'end': 19}

In [ ]:
comb_list2 = []
for i in range(len(df_clean2)):
  combined_dict2 = {}
  combined_dict2["tokens"] = df_clean2.iloc[0,5]
  combined_dict2["annotations"] = [df_anno_dict2[0]]
  comb_list2.append(combined_dict2)

In [ ]:
comb_list2[:1]

[{'tokens': ['Reported',
   'lack',
   'of',
   'DNA',
   'is',
   'another',
   'dead',
   'end',
   'in',
   'O.J',
   '.',
   'Simpson',
   'knife',
   'investigation'],
  'annotations': [{'mention': 'DNA',
    'mention_id': None,
    'labels': [''],
    'start': 17,
    'end': 19}]}]

In [ ]:
with open("rufes_test.json", "w") as outfile:
    json.dump(comb_list2, outfile)

## 3. Model Loading

In [ ]:
!gdown 1mNr2q63nPggzfygHXAPyukb44VjoaMaD

Downloading...
From: https://drive.google.com/uc?id=1mNr2q63nPggzfygHXAPyukb44VjoaMaD
To: /content/best_model.pt
100% 865M/865M [00:07<00:00, 112MB/s]


In [ ]:
import torch
model = torch.load('/content/best_model.pt')

In [ ]:
!wget https://raw.githubusercontent.com/sharvi24/Fine-grained-Entity-Typing/main/smaller.json

In [ ]:
import shutil

# shutil.rmtree('/content/training')
shutil.rmtree('/content/test')
shutil.rmtree('/content/sample_data')

In [ ]:
!python /content/train.py

Loading word embeddings from enwiki.cbow.100d.case.txt
Loaded 2161404 word embeddings
^C


In [ ]:
from util import *
get_label_vocab()

In [ ]:
!wget https://github.com/sharvi24/Fine-grained-Entity-Typing/raw/main/rufes_test.json

In [ ]:
!wget https://github.com/sharvi24/Fine-grained-Entity-Typing/raw/main/rufes_train.json

In [ ]:
!python /content/train.py

Loading word embeddings from enwiki.cbow.100d.case.txt
Loaded 2161404 word embeddings
Collect fine-grained entity labels
Building the model
Epoch 0
Batch 0^C


## 4. Converting Test & Predictions for RUFES Scorer

In [ ]:
# Remove data from train that does not have labels from ontology list

In [ ]:
# Reading predictions - a txt file
my_file = open("/content/label_predictions.txt", "r")
  
# reading the file
data = my_file.read()
# replacing end splitting the text when newline ('\n') is seen.
data_into_list = data.split("\n")
clean_data = []
for d in data_into_list:
  d = d.lstrip('[').rstrip(']')
  d = d.replace(",",";")
  clean_data.append(d.replace("'", ""))


my_file.close()

In [ ]:
clean_data[:6]

['ORG.Association; Polity',
 'ORG.Association; Polity',
 'ORG.Association',
 'ORG.Association',
 'ORG.Association',
 'ORG.Association']

In [ ]:
df2.insert(6, 'new-col', clean_data[:9996])
df2

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,new-col,mention_type,confidence_val
0,NIST,NaN,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NaN,,ORG.Association; Polity,NAM,1.0
1,NIST,annotation-0,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,PER;PER.SituationalRole.Suspect,ORG.Association; Polity,NAM,1.0
2,NIST,annotation-1,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,ORG.Association,NOM,1.0
3,NIST,annotation-2,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,WEA;WEA.CloseCombatWeapon;WEA.CloseCombatWeapo...,ORG.Association,NOM,1.0
4,NIST,annotation-3,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,PER;PER.SituationalRole.Suspect,ORG.Association,NAM,1.0
...,...,...,...,...,...,...,...,...,...
9991,NIST,annotation-88,China,20151229_WAPO_fd884454-a9b4-11e5-bff5-905b92f5...,China,GPE;GPE.Country,Document.LegalDocument,NAM,1.0
9992,NIST,annotation-89,Jerome A. Cohen,20151229_WAPO_fd884454-a9b4-11e5-bff5-905b92f5...,Jerome A. Cohen,PER;PER.Writer,Document.LegalDocument,NAM,1.0
9993,NIST,annotation-90,Xi Jinping,20151229_WAPO_fd884454-a9b4-11e5-bff5-905b92f5...,Xi Jinping,PER,Document.LegalDocument,NAM,1.0
9994,NIST,annotation-92,China,20151229_WAPO_fd884454-a9b4-11e5-bff5-905b92f5...,China,GPE;GPE.Country,GPE.Country,NAM,1.0


In [ ]:
# Add predicted labels to df2.entity_types
del df2['entity_types']
df2.rename(columns = {'new-col':'entity_types'}, inplace = True)
df2.head()

,system_run_ID,mention_ID,mention_string,mention_justification,entity_ID,entity_types,mention_type,confidence_val
0,NIST,NaN,DNA,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,NaN,ORG.Association; Polity,NAM,1.0
1,NIST,annotation-0,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,ORG.Association; Polity,NAM,1.0
2,NIST,annotation-1,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,ORG.Association,NOM,1.0
3,NIST,annotation-2,knife,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,knife,ORG.Association,NOM,1.0
4,NIST,annotation-3,O.J. Simpson,20160315_WAPO_55c03f01631ae453d58744906df9ac5b...,O.J. Simpson,ORG.Association,NAM,1.0


In [ ]:
## Write dataframe to .tab file

In [ ]:
df2.loc[1:].to_csv("test_output.tab", sep="\t", header=None)

In [ ]:
# Changes to predictions: 
# 1. separator is a semi-colon not comma
# 2. include a hierarchy, if dot present: add separate + together
# 3. Ordering of labels
# 4. Convert labels to string from list
# 5. Remove labels which are outside of ontology list

In [ ]:
!python score_submission.py -l ../../input/demo/log.txt -r demo ../../input/log_specifications.txt ../../input/demo/gold.tab ../../input/demo/system_output.tab ../../input/demo/scores2/